# Logistic regression. Part 1

---

## Linear and Logistic Regressions: prediction and metrics


Applied Mathematical Modeling in Banking

---

# 1. Dataset overview

Source: https://www.kaggle.com/c/bike-sharing-demand/data

**Dataset description:**

You are provided hourly rental data spanning two years. For this competition, the training set is comprised of the first 19 days of each month, while the test set is the 20th to the end of the month. You must predict the total count of bikes rented during each hour covered by the test set, using only information available prior to the rental period.

**Data Fields:**

- [x] `datetime` - hourly date + timestamp
- [x] `season` - 
    - [x] 1 = spring
    - [x] 2 = summer
    - [x] 3 = fall
    - [x] 4 = winter
- [x] `holiday` - whether the day is considered a holiday
- [x] `workingday` - whether the day is neither a weekend nor holiday
- [x] `weather`
    - [x] 1: Clear, Few clouds, Partly cloudy, Partly cloudy 
    - [x] 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist 
    - [x] 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds 
    - [x] 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
- [x] `temp` - temperature in Celsius
- [x] `atemp` - “feels like” temperature in Celsius
- [x] `humidity` - relative humidity
- [x] `windspeed` - wind speed
- [x] `casual` - number of non-registered user rentals initiated
- [x] `registered` - number of registered user rentals initiated
- [x] `count` - number of total rentals **(It is target variable. We will predict it!)**


In [1]:
data <- read.csv("data/bikes.csv")
head(data)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
,<chr>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<dbl>,<int>,<int>,<int>
1,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16
2,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40
3,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32
4,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13
5,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1
6,2011-01-01 05:00:00,1,0,0,2,9.84,12.880,75,6.0032,0,1,1


Описова статистика факторів:

In [2]:
summary(data)

   datetime             season         holiday          workingday    
 Length:10886       Min.   :1.000   Min.   :0.00000   Min.   :0.0000  
 Class :character   1st Qu.:2.000   1st Qu.:0.00000   1st Qu.:0.0000  
 Mode  :character   Median :3.000   Median :0.00000   Median :1.0000  
                    Mean   :2.507   Mean   :0.02857   Mean   :0.6809  
                    3rd Qu.:4.000   3rd Qu.:0.00000   3rd Qu.:1.0000  
                    Max.   :4.000   Max.   :1.00000   Max.   :1.0000  
    weather           temp           atemp          humidity     
 Min.   :1.000   Min.   : 0.82   Min.   : 0.76   Min.   :  0.00  
 1st Qu.:1.000   1st Qu.:13.94   1st Qu.:16.66   1st Qu.: 47.00  
 Median :1.000   Median :20.50   Median :24.24   Median : 62.00  
 Mean   :1.418   Mean   :20.23   Mean   :23.66   Mean   : 61.89  
 3rd Qu.:2.000   3rd Qu.:26.24   3rd Qu.:31.06   3rd Qu.: 77.00  
 Max.   :4.000   Max.   :41.00   Max.   :45.45   Max.   :100.00  
   windspeed          casual         regi